In [66]:
import json
import numpy as np
import pandas as pd
from pprint import pprint
from pymongo import MongoClient

In [33]:
# Connect to DB

client = MongoClient('localhost', 27017)

db = client['twitter']

collection = db['twitter']

collection.find_one({})

In [ ]:
selected_features = ['followers_count', 'friends_count', 'listedcount', 'favourites_count',
       'verified', 'statuses_count', 'default_profile',
       'default_profile_image', 'bot',
       'retweet_count', 'fav_count', 'total_usrmention',
       'desc_length', 'name_length', 'screennamebot', 'namebot', 'descbot']

In [39]:
# Store tweets in DB

def process_tweet(tweet):
    processed_tweet = {}
    
    processed_tweet['followers_count'] = tweet['user']['followers_count']
    processed_tweet['friends_count'] = tweet['user']['friends_count']
    processed_tweet['listed_count'] = tweet['user']['listed_count']
    processed_tweet['favourites_count'] = tweet['user']['favourites_count']
    processed_tweet['verified'] = tweet['user']['verified']
    processed_tweet['statuses_count'] = tweet['user']['statuses_count']
    processed_tweet['default_profile'] = tweet['user']['default_profile']
    processed_tweet['default_profile_image'] = tweet['user']['default_profile_image']
    processed_tweet['retweet_count'] = tweet['retweet_count']
    processed_tweet['fav_count'] = tweet['favorite_count']
    processed_tweet['total_usrmention'] = len(tweet['entities']['user_mentions'])
    processed_tweet['name_length'] = len(tweet['user']['name'])
    
    if tweet['user']['description'] is None:
        processed_tweet['desc_length'] = 0
    else:
        processed_tweet['desc_length'] = len(tweet['user']['description'])
    
    return processed_tweet

# with open('tweets.jsons', 'r') as f:
#     for line in f:
#         data = json.loads(line)
#         tweet = process_tweet(data)
#         db.twitter.insert_one(tweet)

In [40]:
# Load tweets from DB

cursor = collection.find({})

df = pd.DataFrame(list(cursor))

df = df.drop('_id', axis=1)

In [42]:
# Load the bot classification model

import pickle

with open('model.pkl', 'rb') as f:
    clf = pickle.load(f)

In [60]:
# Use the model to get the predictions

preds = clf.predict(df)

In [69]:
# Check classification results

unique, counts = np.unique(preds, return_counts=True)

positive_count = counts[1]
negative_count = counts[0]

print("Positive count: ", positive_count)
print("Negative count: ", negative_count)

Positive count:  633358
Negative count:  23949


In [ ]:
from kafka import KafkaProducer

KAFKA_TOPIC = 'tweet_stream'

producer = KafkaProducer(value_serializer=lambda v: json.dumps(v).encode('utf-8'),
                         bootstrap_servers=['localhost:9092'], batch_size=500)

# Must send bytes
messages = df.iloc[:100000]

# Send the messages
for m in messages.iterrows():
    producer.send(KAFKA_TOPIC, m[1].to_json())